In [ ]:
import pandas as pd

In [ ]:
#Creation de la dataFrame à l'aide du fichier csv avec Pandas
df=pd.read_csv("observations.observation.csv")
#Fractionnement de la colonne Location en deux Colonne SCR(Systeme de coordonnée de référence) et de Position(Position Géographique)
df[["SCR","Position"]]=df["location"].str.split(";",expand=True)
#suppresion de la colonne location
df.drop(columns=["location"],axis=1,inplace=True)
#suppression de la chaine SRID= dans le champ SCR(Systeme de coordonnée de référence)
df["SCR"]=df["SCR"].str.replace("SRID=","")
#Suppresion de de la chaine Point() dans le champ Position
df["Position"]=df["Position"].str.replace("POINT (","").str.replace(")","")
#Fractionnement de la colonne Position en Deux colonnes (Longitude et Latitude)
df[["Longitude","Latitude"]]=df["Position"].str.split(" ",expand=True)
#Supppresion de la colonne Position
df.drop(columns=["Position"],axis=1,inplace=True)

In [ ]:
""""Traitement de la colonne recorded_at	(enregistrer le)"""
#Changer le type de la colonne recorded_at au prealable oblet en DateTime(Date et Heure)
df["recorded_at"]=pd.to_datetime(df["recorded_at"])
df["Date_Enregistrement"]=df["recorded_at"].dt.date
df["Heure_Enregistrement"]=df["recorded_at"].dt.time
#Suppresion de la colonne recorded_at
df.drop(columns=["recorded_at",],axis=1,inplace=True)

In [ ]:
""""Traitement de la colonne created_at	(enregistrer le)"""
#Changer le type de la colonne created_at au prealable objet en DateTime(Date et Heure)
df["created_at"]=pd.to_datetime(df["created_at"])
df["Date_Stock"]=df["created_at"].dt.date
#Formate de l'heure au Format H:M:S qui étais au préalable decimal
df["Heure_Stock"]=df["created_at"].dt.strftime("%H:%M:%S")
df.drop(columns=["created_at"],axis=1,inplace=True)

In [ ]:
df

In [ ]:
"""traitement de la colonne source"""
#Suppression de la chaine de caractere (telonics-collars) contenu dans la colonne source
df["source"]=df["source"].str.replace("(telonics-collars)","").str.replace(" ","")

In [ ]:
"""Traitement de la colonne additional """
#transformation de la colonne additionnal en dicttionnaire pour recuperer les elements à l'intérieur
df["additional"]=df["additional"].apply(eval)
#creer un dataFrame dont les éléments sont les differents éléments de la colonne additional
additional_dict=pd.json_normalize(df["additional"])

In [ ]:
additional_dict.head(3)

In [ ]:
#creer un dataFrame dont les éléments sont les differents éléments de la colonne additional
additional_dict=pd.json_normalize(df["additional"])
"""Traitement de la nouvelle dataFrame issu de la chaine de caractere de la colonne additional"""
#Renommage des differentes colonnes 
additional_dict.rename(columns={"frequency":"frequence","mortality":"etat_elephant","horizontal_error":"erreur_precision","satellite_count":"nbre_satelite","low_voltage":"etat_batterie"},inplace=True)
#Suppression de la colonne hdop qui represente la qualité de transmission du signal
additional_dict.drop(["hdop"],axis=1,inplace=True)
#transformation du dataframe additional_dict en un dataFrame de Pandas
additional_dict=pd.DataFrame(additional_dict)
#copie du DataFrame additional_disk
infos_sup=additional_dict
infos_sup["etat_elephant"]=infos_sup["etat_elephant"].apply(lambda x: x if not pd.isnull(x) else "")
#suppression des valeurs NaN dans la DataFrame additional_dict
#1. colonne etat_elephant
additional_dict["etat_elephant"]=additional_dict["etat_elephant"].apply(lambda x:x if not pd.isnull(x) else "")
#2. colonne etat_batterie
additional_dict["etat_batterie"]=additional_dict["etat_batterie"].apply(lambda x:x if not pd.isnull(x) else "")
#3. colonne erreur_precision 
additional_dict["erreur_precision"]=additional_dict["erreur_precision"].apply(lambda x:x if not pd.isnull(x) else "")
#4. colonne nbre_satelite
additional_dict["nbre_satelite"]=additional_dict["nbre_satelite"].apply(lambda x:x if not pd.isnull(x) else "")
#concatenation de la DataFrame d'origine avec la DataFrame issu de additionnal
df=pd.concat([df,additional_dict],axis=1)
#suppression du champ additionnal 
df.drop(["additional"],axis=1,inplace=True)

In [ ]:
df

In [ ]:
""""Traitement du champs exclusion_flags """
#Suppression du champs exclusion_flags
df.drop(["exclusion_flags"],axis=1,inplace=True)
df

In [ ]:
"""Traitement de id cad identifiant de chaque position Geographique"""
#Renommer le champ id en id_Position
df.rename({"id":"id_Position"},axis=1,inplace=True)
#Transformer la colonne id_Position en index pour mieux manipuler les differentes Position
df.set_index("id_Position",inplace=True)

In [ ]:
df

In [ ]:
from geopy.geocoders import Nominatim
import geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
from geopy.distance import geodesic
from ipyleaflet import Map,AntPath,MeasureControl
import ipywidgets
from vega_datasets import data as vds

In [ ]:
indy_500_address="4790 W 16th St,Indianapolis,In 46222"
indy_500=Nominatim(user_agent="tutoriel").geocode(indy_500_address)
indy_500

In [ ]:
(indy_500.latitude,indy_500.latitude)

In [ ]:
indy_500.raw

In [ ]:
data="https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/five_tallest_buildings.csv"
buildings=pd.read_csv(data)
buildings

In [ ]:
geolocator=Nominatim(user_agent="tutoriel")

In [ ]:
buildings

In [ ]:
buildings.drop(columns=['location'],axis=1,inplace=True)

In [ ]:
buildings.drop(columns=['latitude'],axis=1,inplace=True)

In [ ]:
buildings.drop(columns=['lattitude'],axis=1,inplace=True)

In [ ]:
buildings

In [ ]:
#Transformer une address en coordonnee geographique Geocoding
geolocator=Nominatim(user_agent="tutoriel")
limiter=RateLimiter(geolocator.geocode,min_delay_seconds=1)
buildings["location"]=buildings["Building"].apply(lambda loc:limiter(loc))
buildings["longitude"]=buildings["location"].apply(lambda loc:loc.longitude)
buildings["latitude"]=buildings["location"].apply(lambda loc:loc.latitude)
buildings

In [ ]:
#transformer des coordonnées latitude et longitude en Adress
airports=vds.airports()
airports.name[0]

In [ ]:
climate.columns

In [ ]:
#Determiner l'addresse à l'aide des longitude et latitude ou encore reverse
airport=Nominatim(user_agent="tutoriel").reverse((airports.latitude[0],airports.longitude[0]))
airport.name

In [ ]:
#calcule de distance entre les points
distance.distance((7.981648,-5.068855),(8.352763958199997,-4.42660942332745))

In [ ]:
distance.geodesic((7.981648,-5.068855),(8.352763958199997,-4.42660942332745))

In [ ]:
dir(geopy)

In [ ]:
dir(distance.extra)

In [ ]:
from geopy import distance

In [ ]:
#start with widgets
start_dropdown=ipywidgets.Dropdown(options=list(airports.name),values=list(airports.name[0]),description="start:",disabled=False)
end_dropdown=ipywidgets.Dropdown(options=list(airports.name),values=list(airports.name[1]),description="end:",disabled=False)
#function to calculate distance from a to b
#display Map to pass from a to b
def getCoordonate(a,b):
    start_latitude=airports[airports.name==a].latitude.values[0]
    start_longitude=airports[airports.name==a].longitude.values[0]
    end_latitude=airports[airports.name==b].latitude.values[0]
    end_longitude=airports[airports.name==b].longitude.values[0]
    start,end=(start_latitude,start_longitude),(end_latitude,end_longitude)

    m=Map(center=(38,-98),zoom=4,scroll_wheel_zoom=True)
    ant_path=AnPath(locations=(start,end))
    m.add_layer(ant_path)
    measure=MeasureControl(position='top-left',active_color="red",primary_length_unit="miles")
    m.add_control(measure)
    display(m)
    return distance.distance(start,end).miles

    #interaction between function and widget

In [ ]:
df

In [ ]:
df.columns


In [ ]:
df.sort_values(by=["Date_Enregistrement","Heure_Enregistrement"],ascending=False,inplace=True)

In [ ]:
df

In [ ]:
df

In [ ]:
df.head(20)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
date_enreg=df["Date_Enregistrement"]
date_enreg["06ef6ced-0225-4869-a95c-a07d94c4d840"]

In [ ]:
!pip list

In [ ]:
df

In [ ]:
definir_periode_elephant(dt)

In [28]:
dt=df[['Longitude','Latitude']]

,Longitude,Latitude
id_Position,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427014907086079,8.352018970311402
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352437999999999
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.42660942332745,8.352763958199997
...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.73817
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181000000001
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.73817


In [29]:
dt.head(20)

,Longitude,Latitude
id_Position,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427014907086079,8.352018970311402
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352437999999999
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.42660942332745,8.352763958199997
c35de014-0895-4fb4-9a3a-c1835abaec98,-4.426599923414983,8.352478830964548
b470282c-7d4d-41e8-9db2-27f435925f9a,-4.426551,8.352501
4a8b5d5a-5306-4283-8057-61270526683e,-4.426304472751305,8.352476712580239
757a84a6-fa0f-4049-ba14-cbc40cb37a8a,-4.426259334368629,8.352426690190853


In [30]:
len(dt.index)

100

In [31]:
dt.tail(5)

,Longitude,Latitude
id_Position,,
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.73817
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181000000001
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.73817
1f8b5c46-f3c9-463e-9992-c33c69aba799,-3.990983553838555,8.738153295434355
8b542a66-4321-43ed-9cc0-c5ebec5b8ce1,-3.987163,8.711058


In [32]:
dt.tail(5)

,Longitude,Latitude
id_Position,,
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.73817
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181000000001
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.73817
1f8b5c46-f3c9-463e-9992-c33c69aba799,-3.990983553838555,8.738153295434355
8b542a66-4321-43ed-9cc0-c5ebec5b8ce1,-3.987163,8.711058


In [33]:
len(dt)

100

In [34]:
#on creer une liste de coordonnee pour determiner la distance entre chaque coordonnee
cordonnee=[]
i=0
for index in dt.index:
    rows=dt.loc[index]
    #Convertir les coordonnées en floatant qui était des chaînes de cara
    if(i<len(dt)):
        cordonnee.append((float(rows["Latitude"]),float(rows["Longitude"])))
        i=i+1
len(cordonnee)

100

In [35]:
#calculer les distance entre toutes les coordonnees
def calcul_distance(cordonnees):
    distance=0
    for i in range(0,len(cordonnees)):
        if(i+1)<len(cordonnees):
            distance=distance+geodesic(cordonnees[i],cordonnees[i+1]).km
    return distance

In [36]:
c=calcul_distance(cordonnee)
len(cordonnee)

100

In [37]:
c=calcul_distance(cordonnee)
c

185.8996436885432

In [38]:
dt

,Longitude,Latitude
id_Position,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427014907086079,8.352018970311402
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352437999999999
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.42660942332745,8.352763958199997
...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.73817
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181000000001
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.73817


In [39]:
cordonnee

[(7.981648, -5.068855),
 (8.352018970311402, -4.427014907086079),
 (8.364625, -4.426962),
 (8.352438, -4.426686),
 (8.352763958199997, -4.42660942332745),
 (8.352478830964548, -4.426599923414983),
 (8.352501, -4.426551),
 (8.35247671258024, -4.426304472751305),
 (8.352426690190853, -4.426259334368629),
 (8.48412797540209, -4.219099072298079),
 (8.647683, -4.034745),
 (8.717238589969712, -4.033812668689754),
 (8.683775, -4.032363),
 (8.653581783520037, -4.029335523425118),
 (8.716015, -4.014232),
 (8.717535, -3.998268),
 (8.637388961537331, -3.997178305142015),
 (8.647673, -3.996218),
 (8.72609, -3.994769),
 (8.738213, -3.991669),
 (8.738477313976512, -3.991664270600189),
 (8.738278, -3.991647),
 (8.738278, -3.991647),
 (8.738299, -3.991637),
 (8.738256, -3.991633),
 (8.738213, -3.991626),
 (8.738235, -3.991626),
 (8.738267, -3.991626),
 (8.738256, -3.991615),
 (8.738259, -3.991613),
 (8.738235, -3.991604),
 (8.738245, -3.991604),
 (8.738245, -3.991604),
 (8.738256, -3.991604),
 (8.7382

In [40]:
df.sort_values(by=["Date_Enregistrement","Heure_Enregistrement"],ascending=False,inplace=True)

In [41]:
#ajouter chaque element date enregistrement
dt["Date_Enregistrement"]=df["Date_Enregistrement"]

C:\Users\MINEF\AppData\Local\Temp\ipykernel_2088\4241464703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt["Date_Enregistrement"]=df["Date_Enregistrement"]


In [168]:
#definir une fonction distance parcourue par jour en kilometre
def distance_par_jour_km(dataframe):
    #conversion du dataframe en numeric
    dist_total=0
    for i in range(1,len(dataframe)):
        prev_long=dataframe["Longitude"].iloc[i-1]
        prev_lati=dataframe["Latitude"].iloc[i-1]
        prev_pos=(prev_lati,prev_long)
        cur_long=dataframe["Longitude"].iloc[i]
        cur_lati=dataframe["Latitude"].iloc[i]
        cur_pos=(cur_lati,cur_long)
        distance=geodesic(prev_pos,cur_pos).km
        #distance total de chaque groupe
        dist_total+=distance
    return dist_total
#Fin de la fonction

In [43]:
#definir une fonction distance parcourue par jour en metre
def distance_par_jour_metre(dataframe):
    #conversion du dataframe en numeric
    dataframe["Longitude"]=pd.to_numeric(dataframe["Longitude"],errors='coerce')
    dataframe["Latitude"]=pd.to_numeric(dataframe["Latitude"],errors='coerce')
    distances=[]
    for i in range(1,len(dataframe)):
        prev_long=dataframe["Longitude"].iloc[i-1]
        prev_lati=dataframe["Latitude"].iloc[i-1]
        prev_pos=(prev_lati,prev_long)
        cur_long=dataframe["Longitude"].iloc[i]
        cur_lati=dataframe["Latitude"].iloc[i]
        cur_pos=(cur_lati,cur_long)
        distance=geodesic(prev_pos,cur_pos).m
        distances.append(distance)
    #distance total de chaque groupe
    dist_total=sum(distances)
    return dist_total
#Fin de la fonction

In [44]:
#Fonction pour mettre les coordonnees dans une list
def put_cord_in_list(dt):
    cordonnee=[]
    i=0
    for index in dt.index:
        rows=dt.loc[index]
        #Convertir les coordonnées en floatant qui était des chaînes de caracteres
        if(i<len(dt)):
            cordonnee.append((float(rows["Latitude"]),float(rows["Longitude"])))
            i=i+1
    return cordonnee
#Fin de la la function

In [45]:
#start function to put coordonate to float
def put_cooronate_to_float(dataframe):
       dataframe["Longitude"]=pd.to_numeric(dataframe["Longitude"],errors='coerce')
       dataframe["Latitude"]=pd.to_numeric(dataframe["Latitude"],errors='coerce')
#stop function

In [46]:
dt=pd.DataFrame(dt)
put_cooronate_to_float(dt)

In [47]:
dt["Latitude"].info()

<class 'pandas.core.series.Series'>
Index: 100 entries, 06ef6ced-0225-4869-a95c-a07d94c4d840 to 8b542a66-4321-43ed-9cc0-c5ebec5b8ce1
Series name: Latitude
Non-Null Count  Dtype  
--------------  -----  
100 non-null    float64
dtypes: float64(1)
memory usage: 5.6+ KB


In [48]:
dt.head(20)

,Longitude,Latitude,Date_Enregistrement
id_Position,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15
c35de014-0895-4fb4-9a3a-c1835abaec98,-4.426600,8.352479,2024-02-15
b470282c-7d4d-41e8-9db2-27f435925f9a,-4.426551,8.352501,2024-02-14
4a8b5d5a-5306-4283-8057-61270526683e,-4.426304,8.352477,2024-02-15
757a84a6-fa0f-4049-ba14-cbc40cb37a8a,-4.426259,8.352427,2024-02-15


In [49]:
#distance parcourue par jour en Kilometre qui est une Serie avec comme index la Date_Enregistrement et comme colone O(par defaut)
ds=dt.groupby("Date_Enregistrement").apply(distance_par_jour_km)
#distance parcourue par jour en metre qui est une Serie avec comme index la Date_Enregistrement et comme colone O(par defaut)
dk=dt.groupby("Date_Enregistrement").apply(distance_par_jour_metre)

In [50]:
#convertir la serie en DataFrame pour le renommer
dv=pd.DataFrame(ds)
#convertir la Serie en DataFrame pour le renommer
dw=pd.DataFrame(dk)

In [51]:
#renommer la colonne 0 par "Distance Parcoure en Km
dv.rename(columns={0:"Distance Parcoure en Km"},inplace=True)
#renommer la colonne 0 par "Distance Parcoure en Metre
dw.rename(columns={0:"Distance Parcourue en Metre"},inplace=True)

In [52]:
dv

,Distance Parcoure en Km
Date_Enregistrement,
2024-02-05,0.000000
2024-02-06,0.000000
2024-02-07,0.039876
2024-02-08,0.029269
2024-02-09,0.095305
2024-02-10,4.505496
2024-02-11,11.322534
2024-02-12,0.044100
2024-02-13,0.022630


In [53]:
dw

,Distance Parcourue en Metre
Date_Enregistrement,
2024-02-05,0.000000
2024-02-06,0.000000
2024-02-07,39.876152
2024-02-08,29.269014
2024-02-09,95.305448
2024-02-10,4505.496060
2024-02-11,11322.533709
2024-02-12,44.100402
2024-02-13,22.629629


In [55]:
dt=pd.DataFrame(dt)
dt["temps"]=df["temps"]

KeyError: 'temps'

In [56]:
df

,source,SCR,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,Date_Stock,Heure_Stock,frequence,etat_elephant,etat_batterie,erreur_precision,nbre_satelite
id_Position,,,,,,,,,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,738686A,4326,-5.068855,7.981648,2024-02-15,14:01:01,2024-02-15,14:01:23,150.4,False,False,7.5,
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,738686A,4326,-4.427014907086079,8.352018970311402,2024-02-15,12:02:55,2024-02-15,12:28:04,150.4,True,,,6.0
757a84a6-fa0f-4049-ba14-cbc40cb37a8a,738686A,4326,-4.426259334368629,8.352426690190853,2024-02-15,06:01:38,2024-02-15,14:01:23,150.4,False,False,,8.0
c35de014-0895-4fb4-9a3a-c1835abaec98,738686A,4326,-4.426599923414983,8.352478830964548,2024-02-15,04:02:34,2024-02-15,14:01:29,150.4,False,False,,6.0
b0ef4fcc-4791-43a0-9937-291d689a9012,738686A,4326,-4.42660942332745,8.352763958199997,2024-02-15,02:02:25,2024-02-15,14:01:28,150.4,False,False,,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5aaf8ead-be54-40b9-9642-2288ae18e4a6,738686A,4326,-3.991604,8.738288000000001,2024-02-07,04:00:36,2024-02-10,07:14:03,150.4,True,False,9.5,
07d97fb1-2696-448b-8560-311e40a201b3,738686A,4326,-3.991572,8.738277999999999,2024-02-07,02:00:35,2024-02-10,07:14:03,150.4,True,False,7.5,
eeffbc66-1182-49fc-9c4f-f711247d923b,738686A,4326,-3.991594,8.738256,2024-02-07,00:01:00,2024-02-10,07:14:03,150.4,True,False,10.5,


In [57]:
type(df.loc["29563d78-2af6-4d2d-b3b9-c1cad789439d"])

pandas.core.series.Series

In [58]:
#definir la periode cad Journee ou Nuit du dataFrame
def definir_periode_elephant(heure):
    import datetime
    if datetime.time(6)<(heure)<datetime.time(18):
        return "Jour"
    else:
        return "Nuit"

In [59]:
#Fonction permettant de determiner la période ou le temps en fonction de l'heure
import datetime
def definir_periode(heure_enregistrer):
    if datetime.time(6)<heure_enregistrer<datetime.time(18):
        return "Jour"
    else:
        return "Nuit"
#Fin de la Fonction

In [60]:
de=pd.DataFrame(df)
#Appelle de la fonction definir temps sur la dataFrame
de["temps"]=de["Heure_Enregistrement"].apply(definir_periode)

In [61]:
dt["Heure_Enregistrement"]=df["Heure_Enregistrement"]

In [62]:
dt["temps"]=dt["Heure_Enregistrement"].apply(definir_periode)

In [63]:
dt.head(20)

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15,14:01:01,Jour
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15,12:02:55,Jour
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05,17:37:54,Jour
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
c35de014-0895-4fb4-9a3a-c1835abaec98,-4.426600,8.352479,2024-02-15,04:02:34,Nuit
b470282c-7d4d-41e8-9db2-27f435925f9a,-4.426551,8.352501,2024-02-14,20:02:29,Nuit
4a8b5d5a-5306-4283-8057-61270526683e,-4.426304,8.352477,2024-02-15,00:03:05,Nuit
757a84a6-fa0f-4049-ba14-cbc40cb37a8a,-4.426259,8.352427,2024-02-15,06:01:38,Jour


In [64]:
dt.tail(20)

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
d7066e79-4370-46ab-a15b-c1350291e96e,-3.991486,8.738149,2024-02-12,10:00:42,Jour
ca58cbd4-3171-4dac-9594-61ad5f7b3750,-3.991476,8.738117,2024-02-11,20:00:37,Nuit
5e95328b-ab9f-451a-a334-50895ebf63f3,-3.991476,8.738149,2024-02-10,18:01:02,Nuit
fa293ec6-1fc1-41cc-b433-31fa20912414,-3.991476,8.738181,2024-02-11,00:00:36,Nuit
8223d468-e34b-4246-a25f-e1ae27edbd91,-3.991476,8.738181,2024-02-12,00:00:36,Nuit
da27aa6d-1339-4670-bf1f-1e9337181b02,-3.991476,8.738184,2024-02-14,00:00:39,Nuit
2ed37598-b03a-4b14-af20-46c88c6194cb,-3.991471,8.738125,2024-02-12,02:00:34,Nuit
c0a857fa-c61f-4764-b14f-bdce33e187c9,-3.991465,8.738127,2024-02-11,16:00:37,Jour
14fd497a-39c1-4c5c-afad-00b49aeb1a37,-3.991465,8.738138,2024-02-14,04:00:36,Nuit


In [65]:
#distance parcourue par dans la journee comme dans la nuit 
data=dt.groupby(["Date_Enregistrement","temps"]).apply(distance_par_jour_km)

In [66]:
data=pd.DataFrame(data)

In [67]:
data.rename(columns={0:"Distance parcourue"},inplace=True)
data.sort_values(by="Date_Enregistrement",ascending=False)

Distance parcourue
Date_Enregistrement temps                    
2024-02-15          Nuit             0.064095
                    Jour            81.829051
2024-02-14          Nuit            64.764672
                    Jour            28.765727
2024-02-13          Nuit             0.017911
                    Jour             0.005033
2024-02-12          Nuit             0.021229
                    Jour             0.030135
2024-02-11          Nuit             0.034402
                    Jour            11.292578
2024-02-10          Nuit             0.030543
                    Jour             4.487321
2024-02-09          Nuit             0.019515
                    Jour             0.083963
2024-02-08          Nuit             0.018215
                    Jour             0.016559
2024-02-07          Nuit             0.023499
                    Jour             0.016609
2024-02-06          Nuit             0.000000
2024-02-05          Jour             0.000000

In [68]:
data

Distance parcourue
Date_Enregistrement temps                    
2024-02-05          Jour             0.000000
2024-02-06          Nuit             0.000000
2024-02-07          Jour             0.016609
                    Nuit             0.023499
2024-02-08          Jour             0.016559
                    Nuit             0.018215
2024-02-09          Jour             0.083963
                    Nuit             0.019515
2024-02-10          Jour             4.487321
                    Nuit             0.030543
2024-02-11          Jour            11.292578
                    Nuit             0.034402
2024-02-12          Jour             0.030135
                    Nuit             0.021229
2024-02-13          Jour             0.005033
                    Nuit             0.017911
2024-02-14          Jour            28.765727
                    Nuit            64.764672
2024-02-15          Jour            81.829051
                    Nuit             0.064095

In [69]:
df

,source,SCR,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,Date_Stock,Heure_Stock,frequence,etat_elephant,etat_batterie,erreur_precision,nbre_satelite
id_Position,,,,,,,,,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,738686A,4326,-5.068855,7.981648,2024-02-15,14:01:01,2024-02-15,14:01:23,150.4,False,False,7.5,
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,738686A,4326,-4.427014907086079,8.352018970311402,2024-02-15,12:02:55,2024-02-15,12:28:04,150.4,True,,,6.0
757a84a6-fa0f-4049-ba14-cbc40cb37a8a,738686A,4326,-4.426259334368629,8.352426690190853,2024-02-15,06:01:38,2024-02-15,14:01:23,150.4,False,False,,8.0
c35de014-0895-4fb4-9a3a-c1835abaec98,738686A,4326,-4.426599923414983,8.352478830964548,2024-02-15,04:02:34,2024-02-15,14:01:29,150.4,False,False,,6.0
b0ef4fcc-4791-43a0-9937-291d689a9012,738686A,4326,-4.42660942332745,8.352763958199997,2024-02-15,02:02:25,2024-02-15,14:01:28,150.4,False,False,,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5aaf8ead-be54-40b9-9642-2288ae18e4a6,738686A,4326,-3.991604,8.738288000000001,2024-02-07,04:00:36,2024-02-10,07:14:03,150.4,True,False,9.5,
07d97fb1-2696-448b-8560-311e40a201b3,738686A,4326,-3.991572,8.738277999999999,2024-02-07,02:00:35,2024-02-10,07:14:03,150.4,True,False,7.5,
eeffbc66-1182-49fc-9c4f-f711247d923b,738686A,4326,-3.991594,8.738256,2024-02-07,00:01:00,2024-02-10,07:14:03,150.4,True,False,10.5,


In [70]:
dt

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15,14:01:01,Jour
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15,12:02:55,Jour
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05,17:37:54,Jour
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
...,...,...,...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.738170,2024-02-13,22:00:55,Nuit
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181,2024-02-12,16:00:36,Jour
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.738170,2024-02-12,18:00:52,Nuit


In [71]:
demon=dt.groupby("Date_Enregistrement").apply(distance_par_jour_km)

In [72]:
demon

Date_Enregistrement
2024-02-05     0.000000
2024-02-06     0.000000
2024-02-07     0.039876
2024-02-08     0.029269
2024-02-09     0.095305
2024-02-10     4.505496
2024-02-11    11.322534
2024-02-12     0.044100
2024-02-13     0.022630
2024-02-14    83.074885
2024-02-15    81.899650
dtype: float64

In [73]:
#Tentative de groupement par semaine
group_by_week=demon.groupby(pd.Grouper(key="Date_Enregistrement",freq="W"))

KeyError: 'The grouper name Date_Enregistrement is not found'

In [ ]:
group_by_week

In [75]:
dt

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15,14:01:01,Jour
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15,12:02:55,Jour
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05,17:37:54,Jour
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
...,...,...,...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.738170,2024-02-13,22:00:55,Nuit
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181,2024-02-12,16:00:36,Jour
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.738170,2024-02-12,18:00:52,Nuit


In [76]:
(demon.index)

Index([2024-02-05, 2024-02-06, 2024-02-07, 2024-02-08, 2024-02-09, 2024-02-10,
       2024-02-11, 2024-02-12, 2024-02-13, 2024-02-14, 2024-02-15],
      dtype='object', name='Date_Enregistrement')

In [77]:
#Make Index to dateTime
def IndexToDatetime(df):
    df.index=pd.to_datetime(df.index)
    return df
#end of the function

In [78]:
dal=IndexToDatetime(demon.copy())

In [79]:
dm= IndexToDatetime(dt.copy()).groupby(pd.Grouper(key="Date_Enregistrement",freq="W")).apply(distance_par_jour_km)

C:\Users\MINEF\AppData\Local\Temp\ipykernel_2088\3084222719.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index=pd.to_datetime(df.index)


DateParseError: Unknown datetime string format, unable to parse: 06ef6ced-0225-4869-a95c-a07d94c4d840, at position 0

In [80]:
dal

Date_Enregistrement
2024-02-05     0.000000
2024-02-06     0.000000
2024-02-07     0.039876
2024-02-08     0.029269
2024-02-09     0.095305
2024-02-10     4.505496
2024-02-11    11.322534
2024-02-12     0.044100
2024-02-13     0.022630
2024-02-14    83.074885
2024-02-15    81.899650
dtype: float64

In [81]:
dt

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15,14:01:01,Jour
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15,12:02:55,Jour
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05,17:37:54,Jour
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
...,...,...,...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.738170,2024-02-13,22:00:55,Nuit
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181,2024-02-12,16:00:36,Jour
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.738170,2024-02-12,18:00:52,Nuit


In [82]:
example=dt.copy()

In [83]:
example=IndexToDatetime(example)

C:\Users\MINEF\AppData\Local\Temp\ipykernel_2088\3084222719.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index=pd.to_datetime(df.index)


DateParseError: Unknown datetime string format, unable to parse: 06ef6ced-0225-4869-a95c-a07d94c4d840, at position 0

In [84]:
example.index

Index(['06ef6ced-0225-4869-a95c-a07d94c4d840',
       'c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b',
       '29563d78-2af6-4d2d-b3b9-c1cad789439d',
       '6110af12-d0f3-49ec-b8b5-ae6f45cfb95e',
       'b0ef4fcc-4791-43a0-9937-291d689a9012',
       'c35de014-0895-4fb4-9a3a-c1835abaec98',
       'b470282c-7d4d-41e8-9db2-27f435925f9a',
       '4a8b5d5a-5306-4283-8057-61270526683e',
       '757a84a6-fa0f-4049-ba14-cbc40cb37a8a',
       '4878aedd-09bf-45f9-aeeb-8369a08ea448',
       '75e279c9-adc8-45a0-8bea-8e2846cf32e7',
       '285e4bce-5266-4c41-9aa4-a500cacdada8',
       '4d9bf27a-12c3-4298-9bb7-0e4028abd911',
       '67aa1c1c-5b0c-4fbb-9452-fa77e16b0460',
       '74a21a64-23a8-40fe-90e4-5f5ecf943830',
       'a08c8f09-397b-4f88-afe6-c056bcb78d4e',
       '001a03e7-79e0-400b-b133-bd0edb418621',
       '5719bf19-e876-4adf-9d34-79a1d361149c',
       'e79d1289-8fce-4910-85a9-c2a63e2996c2',
       '61cd8db3-349d-41af-a076-ec706a147d7e',
       '677ead29-31e5-4cb9-9d89-d077d25adff0',
       '33cf0

In [85]:
demon.sum()

181.0337450694931

In [86]:
g=dt.groupby(by="Date_Enregistrement")

In [87]:
example=(dt.copy()).set_index("Date_Enregistrement")

In [88]:
example.index=pd.to_datetime(example.index)

In [89]:
example.head(len(example.index))

,Longitude,Latitude,Heure_Enregistrement,temps
Date_Enregistrement,,,,
2024-02-15,-5.068855,7.981648,14:01:01,Jour
2024-02-15,-4.427015,8.352019,12:02:55,Jour
2024-02-05,-4.426962,8.364625,17:37:54,Jour
2024-02-14,-4.426686,8.352438,22:01:07,Nuit
2024-02-15,-4.426609,8.352764,02:02:25,Nuit
...,...,...,...,...
2024-02-13,-3.991433,8.738170,22:00:55,Nuit
2024-02-12,-3.991422,8.738181,16:00:36,Jour
2024-02-12,-3.991411,8.738170,18:00:52,Nuit


In [90]:
example["Longitude"]

Date_Enregistrement
2024-02-15   -5.068855
2024-02-15   -4.427015
2024-02-05   -4.426962
2024-02-14   -4.426686
2024-02-15   -4.426609
                ...   
2024-02-13   -3.991433
2024-02-12   -3.991422
2024-02-12   -3.991411
2024-02-09   -3.990984
2024-02-10   -3.987163
Name: Longitude, Length: 100, dtype: float64

In [91]:
a=example["Longitude"]

In [92]:
a=pd.DataFrame(a)

In [93]:
a
def sume(df):
    d=[]
    for i in range(1,len(df)):
        d.append(df["Longitude"].iloc[i-1])
    return d
b=sume(a)
c=sum(b)
c

-400.1548284549417

In [155]:
#distance parcourue par Jours
#example1=example.groupby(pd.Grouper(level="Date_Enregistrement",freq="w")).apply(distance_par_jour_km)
example1=example.groupby(pd.Grouper(level="Date_Enregistrement",freq="w",sort=True)).get_group("2024-02-11")
example2=example.groupby(pd.Grouper(level="Date_Enregistrement",freq="w"))
example2.size()

Date_Enregistrement
2024-02-11    61
2024-02-18    39
Freq: W-SUN, dtype: int64

In [96]:

#example3=example.groupby(pd.Grouper(level="Date_Enregistrement",freq="MOI")).apply(distance_par_jour_km)
example2.shape

(39, 4)

In [97]:
#Fonction qui convertir la serie obtenu apres le Groupby en DataFrame renomme la colonne en Distance en Kilometre
def add_column_dist_km(serie):
    dataframe=pd.DataFrame(serie)
    dataframe.rename(columns={0:"Distance_Parcourue_Km"},inplace=True)
    return dataframe

In [98]:
#Fonction qui convertir la serie obtenu apres le Groupby en DataFrame renomme la colonne en Distance en Kilometre
def add_column_dist_m(serie):
    dataframe=pd.DataFrame(serie)
    dataframe.rename(columns={0:"Distance_Parcourue_Metre"},inplace=True)
    return dataframe

In [99]:
distance_s=add_column_dist_km(example1)

In [188]:
example1.index=pd.to_datetime(example1.index)

In [182]:
convert_index(example2)

In [184]:
example1.index

DatetimeIndex(['2024-02-05', '2024-02-06', '2024-02-07', '2024-02-07',
               '2024-02-07', '2024-02-07', '2024-02-07', '2024-02-07',
               '2024-02-07', '2024-02-07', '2024-02-07', '2024-02-07',
               '2024-02-07', '2024-02-07', '2024-02-08', '2024-02-08',
               '2024-02-08', '2024-02-08', '2024-02-08', '2024-02-08',
               '2024-02-08', '2024-02-08', '2024-02-08', '2024-02-08',
               '2024-02-08', '2024-02-08', '2024-02-09', '2024-02-09',
               '2024-02-09', '2024-02-09', '2024-02-09', '2024-02-09',
               '2024-02-09', '2024-02-09', '2024-02-09', '2024-02-09',
               '2024-02-09', '2024-02-09', '2024-02-10', '2024-02-10',
               '2024-02-10', '2024-02-10', '2024-02-10', '2024-02-10',
               '2024-02-10', '2024-02-10', '2024-02-10', '2024-02-10',
               '2024-02-10', '2024-02-11', '2024-02-11', '2024-02-11',
               '2024-02-11', '2024-02-11', '2024-02-11', '2024-02-11',
      

In [189]:
dt

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15,14:01:01,Jour
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15,12:02:55,Jour
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05,17:37:54,Jour
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
...,...,...,...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.738170,2024-02-13,22:00:55,Nuit
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181,2024-02-12,16:00:36,Jour
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.738170,2024-02-12,18:00:52,Nuit


In [190]:
dt1=pd.DataFrame(dt)

In [191]:
dt1

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
id_Position,,,,,
06ef6ced-0225-4869-a95c-a07d94c4d840,-5.068855,7.981648,2024-02-15,14:01:01,Jour
c1ab4f12-25a4-4a6f-a9b4-e91a4719f35b,-4.427015,8.352019,2024-02-15,12:02:55,Jour
29563d78-2af6-4d2d-b3b9-c1cad789439d,-4.426962,8.364625,2024-02-05,17:37:54,Jour
6110af12-d0f3-49ec-b8b5-ae6f45cfb95e,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
b0ef4fcc-4791-43a0-9937-291d689a9012,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
...,...,...,...,...,...
68245e89-1e64-4919-9365-77fff468f86b,-3.991433,8.738170,2024-02-13,22:00:55,Nuit
3093d631-8591-4715-8826-c6fde64d7f2c,-3.991422,8.738181,2024-02-12,16:00:36,Jour
616e1bc7-590b-454e-89ba-e27e43f48c32,-3.991411,8.738170,2024-02-12,18:00:52,Nuit


In [241]:
dt1.set_index("Date_Enregistrement",drop=False,inplace=True)

In [242]:
dt1.index=pd.DatetimeIndex(dt1.index)

In [263]:
dt1

,Longitude,Latitude,Date_Enregistrement,Heure_Enregistrement,temps
Date_Enregistrement,,,,,
2024-02-15,-5.068855,7.981648,2024-02-15,14:01:01,Jour
2024-02-15,-4.427015,8.352019,2024-02-15,12:02:55,Jour
2024-02-05,-4.426962,8.364625,2024-02-05,17:37:54,Jour
2024-02-14,-4.426686,8.352438,2024-02-14,22:01:07,Nuit
2024-02-15,-4.426609,8.352764,2024-02-15,02:02:25,Nuit
...,...,...,...,...,...
2024-02-13,-3.991433,8.738170,2024-02-13,22:00:55,Nuit
2024-02-12,-3.991422,8.738181,2024-02-12,16:00:36,Jour
2024-02-12,-3.991411,8.738170,2024-02-12,18:00:52,Nuit


In [410]:
dt2=dt1.groupby(pd.Grouper(level="Date_Enregistrement",freq="2D",sort=True))

In [396]:
dt6=dt1.groupby(pd.Grouper(level="Date_Enregistrement",freq="D",sort=True)).apply(distance_par_jour_km).sum()
dt6

181.0337450694931

In [395]:
for nom_groupe,groupe in dt2:
    print(groupe["Longitude"].tolist())

[-4.426962, -3.991613, -3.991647, -3.991637, -3.991633, -3.991626, -3.991604, -3.991604, -3.991594, -3.991594, -3.991594, -3.991572, -3.991572, -3.991497, -3.991669, -3.991626, -3.991615, -3.991604, -3.991594, -3.991594, -3.991583, -3.991583, -3.991572, -3.991562, -3.991562, -3.991562, -3.991647, -3.991626, -3.991604, -3.991604, -3.991604, -3.991594, -3.991594, -3.991583, -3.991572, -3.99154, -3.99154, -3.990983553838555, -3.994769, -3.991664270600189, -3.991599, -3.991594, -3.991594, -3.991594, -3.991486, -3.991486, -3.991476, -3.991444, -3.987163, -4.033812668689754, -4.032363, -4.014232, -3.991594, -3.991528, -3.991519, -3.991509, -3.991497, -3.991497, -3.991476, -3.991476, -3.991465]
[-3.991508, -3.991508, -3.991508, -3.991497, -3.991486, -3.991476, -3.991471, -3.991454, -3.991454, -3.991433, -3.991422, -3.991411, -3.991543, -3.991529, -3.991519, -3.991519, -3.991508, -3.991508, -3.991508, -3.991508, -3.991433, -4.426686, -4.426551, -4.219099072298079, -4.034745, -4.029335523425118

In [386]:
def distance_par_jour_km(dataframe):
    #conversion du dataframe en numeric
    dataframe["Longitude"]=pd.to_numeric(dataframe["Longitude"],errors='coerce')
    dataframe["Latitude"]=pd.to_numeric(dataframe["Latitude"],errors='coerce')
    distances=[]
    for i in range(1,len(dataframe)):
        prev_long=dataframe["Longitude"].iloc[i-1]
        prev_lati=dataframe["Latitude"].iloc[i-1]
        prev_pos=(prev_lati,prev_long)
        cur_long=dataframe["Longitude"].iloc[i]
        cur_lati=dataframe["Latitude"].iloc[i]
        cur_pos=(cur_lati,cur_long)
        distance=geodesic(prev_pos,cur_pos).km
        distances.append(distance)
    #distance total de chaque groupe
    dist_total=sum(distances)
    return dist_total

In [411]:
distances=[]
for nom_groupe,groupe in dt2:
    grouped=groupe.groupby(level="Date_Enregistrement")
    distances.append(grouped.apply(distance_par_jour_km).sum())
distances

[0,
 0.06914516609615641,
 4.600801508226737,
 11.36663411072203,
 83.09751430370619,
 81.899649980742]

In [412]:
sum(distances)

181.0337450694931

In [322]:
dt2.size()

Date_Enregistrement
2024-02-05     1
2024-02-06     1
2024-02-07    12
2024-02-08    12
2024-02-09    12
2024-02-10    11
2024-02-11    12
2024-02-12    12
2024-02-13     9
2024-02-14    12
2024-02-15     6
Freq: D, dtype: int64

In [373]:
def distance_geodesique_par_groupe_unique(groupe):
  """
  Calcule la distance géodésique entre chaque point d'un groupe d'un DataFrameGroupBy.

  Args:
    groupe (pandas.DataFrame): Un DataFrame représentant un groupe du DataFrameGroupBy.

  Returns:
    pandas.DataFrame: Une nouvelle DataFrame avec la distance géodésique entre chaque point du groupe.
  """

  # On extrait les colonnes de longitude et de latitude
  longitude = groupe["Longitude"]
  latitude = groupe["Latitude"]

  # On crée une liste vide pour stocker les distances
  distances = []

  # On calcule la distance géodésique entre chaque point du groupe et le premier point
  for i in range(len(longitude) - 1):
    distance = geodesic((longitude[i], latitude[i]), (longitude[i + 1], latitude[i + 1]))
    distances.append(distance.km)
  # On crée une nouvelle DataFrame avec les distances calculées
  distance_total=sum(distances)
  return  distance_total

In [377]:
for nom_groupe,groupe in dt2:
    #groupe=pd.DataFrame(groupe[["Longitude","Latitude","temps"]])
    print(distance_geodesique_par_groupe_unique(groupe))
    

86.24751923283311
375.8432758318619


C:\Users\MINEF\AppData\Local\Temp\ipykernel_2088\3620649670.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  distance = geodesic((longitude[i], latitude[i]), (longitude[i + 1], latitude[i + 1]))


In [ ]:
for g,s in dt2:
    print(type(s.tolist()))

In [307]:
def distance_geodesique_par_groupe(dataframe_groupby):
  """
  Calcule la distance géodésique entre chaque groupe d'un DataFrame groupé.

  Args:
    dataframe_groupby (pandas.DataFrameGroupBy): Un objet DataFrameGroupBy contenant les données GPS groupées.

  Returns:
    pandas.DataFrame: Une nouvelle DataFrame avec la distance géodésique entre chaque groupe.
  """

  # On crée une liste vide pour stocker les distances
  distances = []

  # On parcourt les groupes du DataFrame groupé
  for nom_groupe, groupe in dataframe_groupby:
    # On extrait les longitudes et les latitudes du groupe actuel
    longitudes_groupe = groupe["Longitude"].tolist()
    latitudes_groupe = groupe["Latitude"].tolist()

    # On calcule la distance géodésique entre chaque point du groupe et le premier point
    # du groupe suivant
    for i in range(len(longitudes_groupe) - 1):
      distance = geodesic((longitudes_groupe[i], latitudes_groupe[i]), (longitudes_groupe[i + 1],latitudes_groupe[i + 1]))
      distances.append(distance.km)

  # On crée une nouvelle DataFrame avec les distances calculées
  df_distances = pd.DataFrame(distances, columns=["distance_geodesique"])

  # On ajoute le nom du groupe à la nouvelle DataFrame

  return df_distances

# Exemple d'utilisation

# Supposons que vous avez un DataFrame groupé nommé dataframe_groupby

# Appliquez la fonction distance_geodesique_par_groupe au DataFrame groupé
df_distances = distance_geodesique_par_groupe(dt2)

# Affichez la DataFrame df_distances avec la distance géodésique entre chaque groupe
df_distances.sum()


distance_geodesique    181.720841
dtype: float64